# Mallows Kendall
### A python package to for the Mallows model with top-$k$ and complete rankings 

We present methods for working with the Mallows Model (MM), the best-known distribution for permutations. Also, we consider operations for partial rankings (top-$k$ rankings) as well as complete rankings. Cite as 

Fabien Collas, Ekhine Irurozki (2020). Concentric mixtures of Mallows models for top-k rankings: sampling and identifiability. In ArXiv.

In [1]:
import os 
import mallows_kendall as mk
import numpy as np 
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd 


# Preliminaries

## Permutations

**Generation** The most basic function consists in generating a permutation $\sigma$ from $S_n$, the set of all permutations. The permutations are coded as vectors of the first $n$ natural numbers (beginning with 0 here in Python) where each item appears once and once only.   
Complete permutations can be defined here as `ndarrays` with dtype `int` or `list` of integers as well.  
They can be defined by hand as follows:

In [2]:
perm1 = np.array([3,1,2,0,4])
perm1

array([3, 1, 2, 0, 4])

They can also be randomly generated as follows ($n$ being the length of the permutation): 

In [3]:
n=5
perm2 = np.random.permutation(n)
perm2

array([2, 0, 1, 3, 4])

**Operations** Two permutations can be composed and the result is a permutation. This operation is not commutative in general.

In [4]:
mk.compose(perm1, perm2)

array([2, 3, 1, 0, 4])

The inverse of a permutation can also be obtained with:

In [5]:
mk.inverse(perm1)

array([3, 1, 2, 0, 4])

## Distances

The Kendall's-$\tau$ distance $d(\sigma, \pi)$ counts the number of pairwise disagreements beetween $\sigma$ and $\pi$, i.e., the number of pairs of positions in which the items are in a particular order in $\sigma$, and the reverse order in $\pi$. The Kendall's-$\tau$ distance between two permutations can be computed as follows: 

In [6]:
mk.kendall_tau(perm1, perm2)

3

If only one permutation is given in input, it will be assumed that the second permutation is the identity permutation, $e = (1, 2, \dotsc, n)$.

In [7]:
mk.kendall_tau(perm1)

5

The maximum value of the kendall's-$\tau$ distance between two permutation of length $n$ is $\frac{n(n-1)}{2}$. It is possible to get this value using the following function: 

In [8]:
mk.max_dist(n)

10

For each $\sigma \in S_n$ there exists a unique vector, called inversion vector of $\sigma$ with regards to the Kendall's-$\tau$ distance, denoted $V(\sigma) = (V_1(\sigma), \dotsc, V_{n-1}(\sigma))$ and such that $d(\sigma) = \sum_{j=1}^{n-1}V_j(\sigma)$.   
The $j^{th}$ element $V_j(\sigma)$ is given by: $V_j(\sigma) = \sum\limits_{i=j+1}^{n}\mathbb{1}_{\sigma(i) < \sigma(j)}$, $\forall j \in \{1, \dotsc, n-1\}$. It follows that $0 \le V_j(\sigma) \le n-j$. 

Finally there is a bijection between each $\sigma \in S_n$ and each possible $V(\sigma)$ inversion vector. In the package the conversion from one form to another is done using the two following functions:  

- From $\sigma$ to $V(\sigma)$ (here $V$ is of the same length $n$ as the permutation , thus the final $n^{th}$ element of $V$ will always equal $0$) :

In [9]:
V_perm1 = mk.ranking_to_v(perm1)
V_perm1

array([3, 1, 1, 0, 0])

- From $V(\sigma)$ to $\sigma$ :

In [10]:
mk.v_to_ranking(V_perm1, n)

array([3., 1., 2., 0., 4.])

This package also allows to sample a permutation with a given number of inversions, i.e., at a given distance, where all the posible permutations have the same probability of being generated

In [11]:
mk.random_perm_at_dist(n, 2, mk.num_perms_at_dist(n))


array([0., 1., 3., 4., 2.])

*note*: We point out that this notation is taken from Meilǎ, M., Phadnis, K., Patterson, A., & Bilmes, J. (2007). Consensus ranking under the exponential model. In *Proceedings of the 23rd Conference on Uncertainty in Artificial Intelligence, UAI 2007* (pp. 285–294). Note thst the definition of the inversion vector difers from the one in the original paper of MM Fligner, M. A., & Verducci, J. S. (1986). Distance based ranking models. Journal of the Royal Statistical Society, 48(3), 359–369.
This different versions imply different expressions for the MM which will affect the capability of the MM to be used for partial permutations.

# Mallows Model (MM) for complete permutations

The Mallows Model (MM) is an exponential family of probability models for permutation data. We consider $\sigma$ to be a ranking. Formally, the MM using the Kendall's-$\tau$ distance is expressed as follows: 
$$p(\sigma)=\dfrac{\exp(-\theta d(\sigma \sigma_0^{-1}))}{\psi(\theta)}$$ 
where $\psi(\theta) =  \prod_{\substack{j=1}}^{n-1} \frac{1-\exp(-\theta(n-j+1))}{1 - \exp(-\theta)}$.  
$\sigma_0$ represents the central permutation and is the mode of the distribution iff the dispersion parameter $\theta > 0$. In this case, the greater the distance of a permutation to $\sigma_0$ the lower is its probability (it decreases exponentially). The dispersion parameter $\theta$  controls the speed of this fall.



In [12]:
def prob_MM(sigma, sigma_0, theta): 
    
    sigma_0_inv = mk.inverse(sigma_0)
    
    psi = np.prod(np.array([(1 - np.exp(( - n + j ) * theta))/(1 - np.exp(-theta)) for j in range(n-1)]))
    
    dist = mk.kendall_tau( mk.compose(sigma, sigma_0_inv) )
    
    return np.exp( - theta *  dist ) / psi
    
n = 5    
sigma = np.array([3,1,2,0,4])
sigma_0 = np.array(range(5))
theta = 0.1

print(prob_MM(sigma, sigma_0, theta))

0.008161793281714687


**Sampling** This package includes a sampler based on the factorization of the Kendall's-$\tau$ distance. In the later sections we will also present how this can be adapted to top-$k$ rankings. This differs to the classical sampling, usually done using the Repeated Insertion Model (RIM) and which can not be extended to top-$k$ rankings.


In [12]:
mk.sampling_mm(m=4,n=5,theta=1.5)

array([[0., 2., 3., 4., 1.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [2., 0., 1., 3., 4.]])

Note that in the package the sampling functions generates the samples considering $\sigma_0 = e$, identity permutation by default. But any other central permutation can be given as a parameter.  
In practice, we can draw a sample from a MM as follows:

In [13]:
mk.sampling_mm(m=4, n=5, theta=1.5, s0=np.array([4,3,2,1,0]))

array([[4., 3., 2., 1., 0.],
       [4., 2., 1., 3., 0.],
       [4., 2., 1., 3., 0.],
       [4., 3., 2., 1., 0.]])

Usually, the MM is given with the following equivalent expression
$$p(\sigma) \propto \phi^{d(\sigma,\sigma_0)},$$
where $\phi^{d(\sigma,\sigma_0)}=\exp(-\theta d(\sigma,\sigma_0))$ and 
for which the dispersion parameter is denoted as $\phi\in [0,1]$. We can relate the two dispersion parameters, $\theta$ and $\phi$, of the two expressions as  $\phi=-\log(\theta)$. In this package, we can use also this second formulation by specifying parameter `phi` instead of `theta`. This functionallity holds for most funstions. The sampling, for example, is done then as follows:

In [8]:
mk.sampling_mm(m=4,n=5,phi=.5)

array([[1., 4., 3., 0., 2.],
       [0., 2., 1., 4., 3.],
       [0., 1., 2., 3., 4.],
       [0., 1., 4., 2., 3.]])

and the  trasformation between the two is as follows

In [11]:
mk.theta_to_phi(0.7), mk.phi_to_theta(0.7)

(0.4965853037914095, 0.35667494393873245)

**Expected distance** The expected value of Kendall's-$\tau$ distance under the MM is given by: 
$$\mathbb{E}[D] = \frac{n \cdot \exp(-\theta)}{1 - \exp(-\theta)} - \sum_{j=1}^n\frac{j \cdot \exp(-j \theta)}{1 - \exp(-j \theta)}$$

In [42]:
theta_mm = 0.7
sample_mm = mk.sampling_mm(m=4,n=5,theta=1.5)
expected_dist = mk.expected_dist_mm(n, theta_mm)
expected_dist

2.4578024409695693

**Variance** The variance of Kendall's-$\tau$ distance under the MM can be expressed as follows: 
$$\mathbb{V}[D] = \dfrac{ n \cdot \exp(-\theta) }{ (1 - \exp(-\theta))^2 } - \sum_{j=1}^n \dfrac{ j^2\exp(-j \theta) }{ (1 - \exp(-j \theta))^2 }$$

In [43]:
variance_dist = mk.variance_dist_mm(n, theta_mm)
variance_dist

2.763301117320907

**Learning** Then Borda algorithm allows to approximate the central permutation:  

In [44]:
mk.borda(sample_mm)

array([0, 1, 2, 3, 4])

The function `fit_mm` returns an approximation to the MLE of the parameters $\sigma_0$ and $\theta$. The consensus $\sigma_0$ is approximated with the well known Borda count algorithm. 

In [45]:
sigma_borda, phi_mm_mle = mk.fit_mm(sample_mm)
print(sigma_borda)
print(phi_mm_mle)

[0 1 2 3 4]
0.11776019465584917


MM is also often defined as: $$p(\sigma)=\dfrac{\phi^{d(\sigma \sigma_0^{-1})}}{\prod_{\substack{j=1}}^{n-1} \frac{1-\phi^{(n-j+1)}}{1 - \phi}}$$ which implies that $\phi = \exp(-\theta)$. The next function allows to obtain $\theta$ given $\phi$.

*Remark:* $\phi$ is in $[0,1]$ here.

In [46]:
theta_mm_mle = mk.phi_to_theta(phi_mm_mle)
theta_mm_mle

2.1391049710150876

# Mallows Model for Top-$k$ rankings

**Definition** A top-$k$ ranking ($k \le n$) $\sigma$ is a ranking $\sigma = (\sigma(1), \sigma(2), \dotsc, \sigma(n))$ for which only the first $k$ ranks  are known. Hence ranks of items $i$ such that $\sigma(i) \le k$.

An example of top-5 ranking (with $n=10$) with the package would be:

In [47]:
alpha = np.array([ 4.,  0., np.NaN, 1., np.NaN, 3., np.NaN, 2., np.NaN, np.NaN])
alpha

array([ 4.,  0., nan,  1., nan,  3., nan,  2., nan, nan])

**Sampling** In this part the sampling of the top-$k$ rankings is discussed.  
Perhaps the most natural idea to generate top-$k$ rankings would be to generate the full rankings, using the Repeated Insertion Model for example, and to cut these obtained permutations after position $k$. This is possible, yet it seems computationally non-optimal with a complexity of $O(n^2)$.    

Here we adapt the methods of previous sections to top-$k$ rankings. This method is based on sampling partially the inversion vectors. 

In practice in the package, a top-$k$ ranking can be sampled as follows: 

In [48]:
m = 15
n = 10 
k = 5
phi = 0.9 
sigma_0 = np.array(range(10))
sigma = np.random.permutation(n)
sample_top_k = mk.sampling_top_k_rankings(m, n, k, phi = phi, s0=sigma_0)
sample_top_k

array([[ 3., nan, nan,  1.,  0.,  4., nan, nan, nan,  2.],
       [ 3.,  2., nan, nan,  1., nan, nan, nan,  0.,  4.],
       [ 1.,  4.,  2., nan, nan, nan,  3., nan,  0., nan],
       [ 0.,  1., nan,  4., nan, nan, nan,  3.,  2., nan],
       [nan, nan, nan,  0.,  3.,  1., nan, nan,  2.,  4.],
       [ 2., nan,  3.,  4., nan, nan, nan, nan,  0.,  1.],
       [nan, nan,  1.,  3., nan,  2.,  4.,  0., nan, nan],
       [ 1.,  2., nan, nan,  3., nan,  0., nan, nan,  4.],
       [ 3., nan, nan,  1., nan,  2.,  0.,  4., nan, nan],
       [ 0., nan,  3.,  1., nan,  2., nan, nan,  4., nan],
       [nan, nan,  2.,  3., nan,  4., nan, nan,  0.,  1.],
       [ 4.,  0., nan, nan,  3.,  2., nan, nan, nan,  1.],
       [nan,  2.,  1.,  3.,  4., nan, nan, nan, nan,  0.],
       [ 2.,  0., nan, nan,  3.,  1., nan,  4., nan, nan],
       [ 0.,  3., nan,  2., nan,  1., nan,  4., nan, nan]])

**Distance** In order to compare top-$k$ rankings, it is possible to use an extension of the classical Kendall's-$\tau$ denoted as $p$-parametrized Kendall's-$\tau$ distance (Fagin et al., 2003). 

In practice we can use it, chosing the $p$ parameter in $[0,1]$, as follows: 

In [49]:
mk.p_kendall_tau(sample_top_k[0], sample_top_k[1], k=5, p=0)

12

In [50]:
mk.p_kendall_tau(sample_top_k[0], sample_top_k[1], k=5, p=1)

14

**Expected distance** The expected value of Kendall's-$\tau$ distance under the MM is given by: 
$$\mathbb{E}[D] = \frac{k \cdot \exp(-\theta)}{1 - \exp(-\theta)} - \sum_{j=n-k+1}^n\frac{j \cdot \exp(-j \theta)}{1 - \exp(-j \theta)}$$

In [51]:
expected_dist = mk.expected_dist_top_k(n,k,theta_mm)
expected_dist

4.732931959531709

**Variance** The variance of Kendall's-$\tau$ distance under the MM can be expressed as follows: 
$$\mathbb{V}[D] = \dfrac{ k \cdot \exp(-\theta) }{ (1 - \exp(-\theta))^2 } - \sum_{j=n-k+1}^n \dfrac{ j^2\exp(-j \theta) }{ (1 - \exp(-j \theta))^2 }$$

In [52]:
variance_dist = mk.variance_dist_top_k(n,k,theta_mm)
variance_dist

8.391580684957216

# Generalized Mallows Model (GMM) for complete permutations

The Generalized Mallows Model (GMM) is an extension of the Mallows Model, for which there are $n-1$ dispersion parameters $\theta_j$ ($1 \le j < n$), each affecting a particular position of the permutation.   
Formally, the GMM under Kendall's-$\tau$ distance is expressed as follows: 

$$p(\sigma)=\dfrac{\exp(\sum_{j=1}^{n-1}-\theta_j V_j(\sigma\sigma_0^{-1}))}{\psi(\theta)}$$
where $\psi(\theta) = \prod_{j=1}^{n-1} \psi_j(\theta_j) = \prod_{j=1}^{n-1} \frac{1-\exp(-\theta_j(n-j+1))}{1 - \exp(-\theta_j)}$

In [53]:
def prob_GMM(sigma, sigma_0, theta):
    
    n = len(sigma)
        
    sigma_0_inv = mk.inverse(sigma_0)
    
    V = mk.ranking_to_v(mk.compose(sigma, sigma_0_inv))
    
    psi = np.prod(np.array([(1 - np.exp(( - n + j ) * theta[j]))/(1 - np.exp(-theta[j])) for j in range(n-1)]))
    
    return np.exp( np.sum ( [ -theta[j] * V[j] for j in range(n-1) ] ) ) / psi 
    
    
sigma = np.array([3,1,2,0,4])
sigma_0 = np.array(range(5))
theta = [0.5,0.2,0.6,0.3]

print(prob_GMM(sigma, sigma_0, theta))

0.00439275005307042


**Sampling** It is also possible to sample the Generalized Mallows Models (GMM), using the same process as for the MM only with a change in the probability of $V_j$ as follows:

$$p(V_j(\sigma\sigma_0^{-1}) = r) = \frac{\exp(-\theta r)}{\psi_j(\theta)}, \; \forall r \in {0, \dotsc, n-j}$$

In the package it is quite similar as for the classical Mallows Models, only that the dispersion parameter will be given as a list. 

In [54]:
m = 4000
n = 5
theta_gmm = [0.5,0.2,0.6,0.3]
identity = np.array(range(n))
sample_gmm = np.array(mk.sampling_gmm(m,theta_gmm))

The expected value of each term $V_j(\sigma)$, where $\sigma$ is a random Mallows permutation, is expressed as follows:
$$ \mathbb{E}[V_j] = \frac{\exp(-\theta_j)}{1 - \exp(-\theta_j)} - \frac{(n-j+1)\exp(-\theta_j(n-j+1))}{1 - \exp(-\theta_j(n-j+1))}, \;\forall j \in \{1, \dotsc, n-1\} $$
And the values are computed as follows:

In [55]:
expected_v_gmm = mk.expected_v(n, theta_gmm)
print(expected_v_gmm)

[1.09436663 1.25279068 0.62226834 0.42555748]


The expected value of Kendall's-$\tau$ distance under the GMM is then given by: 
$$\mathbb{E}[D] = \sum_{j=1}^{n-1} \mathbb{E}[V_j]$$

In [56]:
expected_dist_gmm = np.sum(expected_v_gmm)
print(expected_dist_gmm)

3.394983134634349


The variance of each term $V_j(\sigma)$ under the GMM can be expressed as follows: 
$$\mathbb{V}[V_j] = \dfrac{ \exp(-\theta_j) }{ (1 - \exp(-\theta_j))^2 } - \sum_{j=1}^n \frac{ (n-j+1)^2 \exp(-(n-j+1) \theta_j) }{ (1 - \exp(-(n-j+1) \theta_j))^2 }, \;\forall j \in \{1, \dotsc, n-1\} $$

In [57]:
variance_v_gmm = mk.variance_v(n, theta_gmm)
print(variance_v_gmm)

[1.48213789 1.20855956 0.56066479 0.24445831]


The expected value of Kendall's-$\tau$ distance under the GMM is then given by: 
$$\mathbb{V}[D] = \sum_{j=1}^{n-1} \mathbb{V}[V_j]$$

In [58]:
variance_dist_gmm = np.sum(variance_v_gmm)
print(variance_dist_gmm)

3.495820543795291


**Learning** An approximation to the MLE for $\theta$ and $\sigma_0$ given a sample of i.i.d Mallows permutations is as follows: 

In [33]:
sigma_borda, theta_gmm_mle = mk.fit_gmm(sample_gmm)
print(sigma_borda)
print(theta_gmm_mle)

[0 2 1 3 4]
[0.5068516942541228, 0.21164074473119746, 0.6040503591890671, 0.3443631773004668]
